# Some Examples with Calculation: Absolute Vorticity and Temperature Advection

## Imports

In [1]:
import sys
sys.path.append('/home/decker/lab433')

from synoptic import MODEL, display_grids

from datetime import datetime

import numpy as np
import xarray as xr
from metpy.io import GempakGrid
from metpy.units import units
from metpy.plots.declarative import ContourPlot, FilledContourPlot, BarbPlot, MapPanel, PanelContainer
import metpy.calc as mpcalc

## Get the Data

In [2]:
gem_file_name = MODEL + 'nam/24112200_nam211.gem'
gem_file = GempakGrid(gem_file_name)

In [3]:
plot_time = datetime(2024, 9, 19, 12)
ht500 = gem_file.gdxarray(parameter='HGHT', date_time=plot_time, level=800)[0]
u500 = gem_file.gdxarray(parameter='UREL', date_time=plot_time, level=800)[0]
v500 = gem_file.gdxarray(parameter='VREL', date_time=plot_time, level=800)[0]

# Attach units
ht500 = ht500 * units('m')
u500 = u500 * units('m/s')
v500 = v500 * units('m/s')

KeyError: 'No grids were matched with given parameters.'

## Calculate Something
Generally, we don't need to do all of the work from scratch. Try `mpcalc.<TAB>` or consult the MetPy documentation for all of the calculations provided by MetPy.

In [ ]:
abs_vort = mpcalc.absolute_vorticity(u500, v500)


## Make a Plot

Let's overlay 500-mb heights (lines; this time in meters) on top of the vorticity (filled contours).

In [ ]:
isohypses = ContourPlot()
isohypses.data = ht500
isohypses.time = plot_time
isohypses.contours = range(0, 7000, 60)
isohypses.linecolor = 'slategray'
isohypses.linestyle = 'solid'
isohypses.clabels = True
isohypses.plot_units = 'm'

fills = FilledContourPlot()
fills.data = abs_vort
fills.time = plot_time
fills.contours = range(0, 40, 4)
fills.scale = 1e5
fills.colormap = 'prism'
fills.colorbar = 'horizontal'

panel = MapPanel()
panel.area = [-120, -74, 22, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'500-mb Heights at {plot_time}'
panel.plots = [fills, isohypses]

pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]
pc.show()

## Example 2: Temperature Advection

### Grab the data
Let's plot temperature advection at 850-mb.

In [ ]:
tmpk = gem_file.gdxarray(parameter='TMPK', date_time=plot_time, level=850)[0]
urel = gem_file.gdxarray(parameter='UREL', date_time=plot_time, level=850)[0]
vrel = gem_file.gdxarray(parameter='VREL', date_time=plot_time, level=850)[0]
hght = gem_file.gdxarray(parameter='HGHT', date_time=plot_time, level=850)[0]

tmpk = tmpk * units('degK')
urel = urel * units('m/s')
vrel = vrel * units('m/s')
hght = hght * units('m')

### Calculate advection

In [ ]:
temp_adv = mpcalc.advection(tmpk, urel, vrel)

### Plot it
When plotting a quantity (like advection) where we care about the positives and negatives and not really the values near zero, a "diverging" colormap often works well.

In [ ]:
isohypses = ContourPlot()
isohypses.data = hght
isohypses.time = plot_time
isohypses.contours = range(0, 400, 3)
isohypses.linecolor = 'dimgray'
isohypses.linestyle = 'solid'
isohypses.clabels = True
isohypses.plot_units = 'dam'

fills = FilledContourPlot()
fills.data = temp_adv
fills.time = plot_time
fills.contours = np.arange(-7.5,7.6).tolist()
fills.scale = 1e4
fills.colormap = 'gist_ncar'
fills.colorbar = 'horizontal'

panel = MapPanel()
panel.area = [-120, -74, 22, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'850-mb Heights and Temp. Adv. at {plot_time}'
panel.plots = [fills, isohypses]

pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]
pc.show()